In [1]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib as plt
%matplotlib inline
import os
import sys
import math
from tensorflow import logging
from tensorflow import gfile
from pprint import pprint
import tensorflow as tf

/home/ubuntu/selfFile/venv/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/selfFile/venv/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/selfFile/venv/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/selfFile/venv/lib/python3.5/site-packages/tensorflow/pyth

In [2]:
'''
    0123:表示用户的编号
    [100, 200, 300, 400, 500]:表示我们的商品的编码
    user-->item: 表示我们用户买过的商品列表
'''
user_item={0: [100, 200, 400], 1: [100, 200], 2: [200, 500], 3: [300, 400, 500]}
pprint(user_item)

{0: [100, 200, 400], 1: [100, 300], 2: [200, 500], 3: [300, 400, 500]}


In [3]:
#计算用户兴趣相似度
def user_sim(user_item):
    '''
       user_item: 表示的是 user-->item 的映射
       item_user: 表示的是 item-->user 的映射
       weight_matrix: 表示用户之间的相似度
    '''
    #做成物品--> 用户的映射
    item_user = {}
    for user, items in user_item.items():
        # 遍历我们所有的物品，然后找到相对应的用户
        for item in items:
            if item not in item_user:
                item_user[item] = set()
            item_user[item].add(user)
    pprint(item_user) 
    
    # 存储我们的用户相似矩阵
    user_sim_matrix = np.zeros((len(user_item.keys()), len(user_item.keys())))
    print("初始化相关矩阵")
    pprint(user_sim_matrix)
    print("user_sim_matrix shape is :", user_sim_matrix.shape)
    for item, users in item_user.items():
        # 对我们的用户进行排列组合(A, B) ==> (A, B), (B, A)
        import itertools
        combination = [list(i) for i in itertools.combinations(users,2)]
#         pprint(combination)
        for i in combination:
            print("要更新的下标是{0}".format(i))
            row, col = i
            user_sim_matrix[row, col] += 1
            user_sim_matrix[col, row] += 1
    print("相关矩阵的最终结果")
    pprint(user_sim_matrix)
    
    # 把我们的矩阵转化为相关的的权重
    matrix_sum = sum(sum(user_sim_matrix) / 2)
    print(matrix_sum)
    weight_matrix = user_sim_matrix / matrix_sum
    
    print("权重矩阵的输出结果：")
    pprint(weight_matrix)
    return weight_matrix

In [4]:
weight_matrix = user_sim(user_item)

{100: {0, 1}, 200: {0, 2}, 300: {1, 3}, 400: {0, 3}, 500: {2, 3}}
初始化相关矩阵
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])
user_sim_matrix shape is : (4, 4)
要更新的下标是[0, 2]
要更新的下标是[0, 3]
要更新的下标是[0, 1]
要更新的下标是[2, 3]
要更新的下标是[1, 3]
相关矩阵的最终结果
array([[0., 1., 1., 1.],
       [1., 0., 0., 1.],
       [1., 0., 0., 1.],
       [1., 1., 1., 0.]])
5.0
权重矩阵的输出结果：
array([[0. , 0.2, 0.2, 0.2],
       [0.2, 0. , 0. , 0.2],
       [0.2, 0. , 0. , 0.2],
       [0.2, 0.2, 0.2, 0. ]])


In [5]:
# 进行推荐系统的实现
def recommend(user, user_item, weight_matrix, k=2):
    '''
        user: 表示我们要向用户user进行商品的推荐
        user_item: user-->item 的映射关系
        weight_matrix: 权重矩阵
        k: 表示我们选取几个人来进行协同推荐
    '''
    # 推荐商品列表{商品： 推荐程度}
    rank_items = {}
    # 这里我们假定所有用户对意见产品的喜欢程度为1
    rvi = 1 
    # 表示我们用户user买过的产品,用来进行商品的筛选，因为买过了，就不再进行推荐
    interacted_items = user_item[user]
    # 找出和我们用户 user 兴趣比较相近的用户
    related_users = weight_matrix[user] # 得到整行数据
    print("找到和用户 user 所有相关的用户")
    # 对我们的用户进行排序
    # 如果我们用户传进来的人数比我们整的人数还多，我们就进行clips
    if k >= len(related_users):
        print("总的人数为", len(related_users))
        k = len(related_users) - 1
    # 找到前 k 个兴趣最相近的下标，此时下标就是用户
    k_related_users = np.argsort(-related_users)[:k]
    print("找到前 {0} 个兴趣最相近的用户".format(k))
    pprint(k_related_users)
    
    for related_user in k_related_users:
        for item in user_item[related_user]:
            # 如果此商品已经买过了，那么我们就忽略
            if item in interacted_items:
                continue
            # 如果我们的商品还没有被买过，那么就有被推荐的可能
            if item not in rank_items.keys():
                rank_items[item] = 0
            rank_items[item] += weight_matrix[user][related_user] * rvi
    print("最终推荐的产品及推荐指数： ")
    pprint(rank_items)
    return rank_items

In [7]:
# inference
recommend(0, user_item, weight_matrix, 4)

找到和用户 user 所有相关的用户
总的人数为 4
找到前 3 个兴趣最相近的用户
array([1, 2, 3])
最终推荐的产品及推荐指数： 
{300: 0.4, 500: 0.4}


{300: 0.4, 500: 0.4}